In [1]:
import warnings

import pandas as pd

from sklearn.linear_model import LogisticRegression

import funciones_auxiliares_hito_4 as aux4
from funciones_auxiliares_hito_2 import preprocesar_y_recodificar_enunciado_uno

# warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('income-db.csv')
df_final = preprocesar_y_recodificar_enunciado_uno(df)

# Recapitulación

Como se mencionó en el hito 3, se realizarán 3 modelos predictivos distintos:

1. Modelo con todas las variables, independiente del p-value encontrado
2. Modelo que afectan significativamente a la variable dependiente (p < 0.05)
3. Modelo que afectan significativamente a la variable dependiente (p < 0.05) y tengan un
coeficiente absoluto > 0.6 ('gender', 'elementary_school', 'university', 'married',
       'others', 'white_collar', 'Not_in_family', 'Other_relative',
       'Own_child', 'Unmarried', 'Wife', 'Asian_Pac_Islander', 'america',
       'asia', 'europe'])

Por lo tanto, en los siguientes apartados se verá la modelación de cada uno de los modelos
estipulados, junto a su desempeño con la métrica de roc-auc score.

**Cabe destacar que previo a realizar el fit del modelo, todas las variables independientes seran
escaladas con un StandardScaler()**

# 1. Modelo completo

In [3]:
df_modelo_completo = df_final.copy()

rocs_modelo_completo = aux4.calcular_roc_cross_val(LogisticRegression(), 10, 'roc_auc',
                                                   df_modelo_completo, 'income')
rocs_modelo_completo.mean()

0.8974375536798401

# 2. Modelo con variable p-value < 0.05

In [4]:
df_modelo_menor_a_005 = df_final[['Own_child', 'elementary_school', 'Other_relative',
       'Unmarried', 'Not_in_family', 'gender', 'others', 'high_school',
       'self_employed', 'fnlwgt', 'capital_gain', 'capital_loss', 'age',
       'hours_per_week', 'separated', 'federal_gov', 'divorced', 'White',
       'widowed', 'white_collar', 'Asian_Pac_Islander', 'university', 'Wife',
       'asia', 'married', 'america', 'europe', 'income']]

rocs_modelo_menor_a_005 = aux4.calcular_roc_cross_val(LogisticRegression(), 10, 'roc_auc',
                                                   df_modelo_menor_a_005, 'income')

rocs_modelo_menor_a_005.mean()

0.8974140936193479

# 3. Modelo con p-value < 0.05 y coefs > 0.6

In [5]:
df_modelo_menor_a_005_y_coef_06 = df_final[['gender', 'elementary_school', 'university', 'married',
       'others', 'white_collar', 'Not_in_family', 'Other_relative',
       'Own_child', 'Unmarried', 'Wife', 'Asian_Pac_Islander', 'america',
       'asia', 'europe', 'income']]

rocs_modelo_menor_a_005_y_coef_06 = aux4.calcular_roc_cross_val(LogisticRegression(), 10, 'roc_auc',
                                                   df_modelo_menor_a_005_y_coef_06, 'income')

rocs_modelo_menor_a_005_y_coef_06.mean()

0.8567990818684967

# Selección del mejor modelo

Los resultados obtenidos fueron los siguientes:

|Modelo|ROC-AUC Score Promedio (Para una validación cruzada de 10-Folds)|
|------|----------------------------------------------------------------|
|Todas las variables|0.89743|
|Variables con p-value < 0.05|0.89741|
|Variables con p-value < 0.05 y coefs > 0.6|0.85680|

Se puede ver que el modelo 1 y el modelo 2 presenten valores ROC-AUC Score muy similares entre si.
Si nos ceñimos solamente a esta métrica, entonces el modelo 1 es el mejor modelo. Sin embargo, 
este modelo ocupa 32 variables independientes. Por otro lado, el modelo 2 ocupa 27 variables independientes (5 menos que el primer modelo). Teniendo en cuenta lo anterior, es posible confirmar 
que el modelo 2 es el mejor modelo de todos. Esto, ya que a pesar de tener un ROC-AUC Score promedio menor que el modelo 1, este es más parsimonioso (utiliza menos variables).

O sea: **El modelo con variable p-value < 0.05 es el mejor modelo**